In [1]:
import onnx
import numpy as np
from qonnx.util.basic import qonnx_make_model
from finn.util.visualization import showInNetron,showSrc
import onnxruntime as rt
from qonnx.util.basic import qonnx_make_model
from onnx.helper import make_tensor_value_info, make_node, make_graph, make_model, make_tensor
from onnx import numpy_helper
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
import finn.core.onnx_exec as oxe
import torch
from brevitas.nn import QuantReLU, QuantIdentity
from finn.transformation.streamline import Streamline
from finn.transformation.streamline.reorder import MoveScalarLinearPastInvariants
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline import RoundAndClipThresholds
from qonnx.core.datatype import DataType
from qonnx.transformation.qcdq_to_qonnx import QCDQToQuant
import finn.core.onnx_exec as oxe

In [2]:
# id_path = "quant_threshold.onnx"
#id_path = "quant_threshold_clip.onnx"
#id_path = "lstm-gate-sigmoid.onnx"
id_path = "lstm-gate-sigmoid-unsigned.onnx"
#id_path = "lstm-gate-tanh.onnx"
#id_path = "quantize-lstm-full-graph-test.onnx"
model = ModelWrapper(id_path)
model = model.transform(QCDQToQuant())
model = model.transform(InferShapes())
model.save("./quant_threshold_transform.onnx") 

In [3]:
showInNetron('./lstm-gate-sigmoid-unsigned.onnx',localhost_url='xirxlabs53')#,localhost_url='xirxlabs53'
#showInNetron('./lstm-gate-tanh.onnx',localhost_url='xirxlabs53')#,localhost_url='xirxlabs53'

Serving './lstm-gate-sigmoid-unsigned.onnx' at http://0.0.0.0:5902


In [4]:
showInNetron('./quant_threshold_transform.onnx',localhost_url='xirxlabs53')#,localhost_url='xirxlabs53'

Stopping http://0.0.0.0:5902
Serving './quant_threshold_transform.onnx' at http://0.0.0.0:5902


In [5]:
#in_X = np.array([0,0,0,0,0,0,0,0,0,0],dtype=np.float32)
in_X = np.array([1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
#in_X = np.array([1,2,3,4,5,-6,-7,-8,-9,-10],dtype=np.float32)
#in_X = np.array([0,0,0,0,0,0,0,0,0,-0.7],dtype=np.float32)
in_X = in_X.reshape([10,1])
in_h_t_1 = np.zeros([20,1],dtype=np.float32).reshape([20,1])
input_dict = {}
input_dict["X"] = in_X
input_dict["h_t-1"] = in_h_t_1

output_dict = oxe.execute_onnx(model, input_dict,return_full_exec_context=True)
output_dict

{'h_t-1': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], dtype=float32),
 'X': array([[ 1.],
        [ 2.],
        [ 3.],
        [ 4.],
        [ 5.],
        [ 6.],
        [ 7.],
        [ 8.],
        [ 9.],
        [10.]], dtype=float32),
 'scan_out': array([[0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294],
        [0.6235294]], dtype=float32),
 'out_m1': array([[27.392155],
        [27.392155],
        [27.392155],


In [6]:
finn_model = model.transform(ConvertQONNXtoFINN())
finn_model.save("./quant_threshold_finn_onnx.onnx") 

In [16]:
showInNetron('./quant_threshold_finn_onnx.onnx',localhost_url='xirxlabs53')#,,localhost_url='xirxlabs53'

Stopping http://0.0.0.0:5902
Serving './quant_threshold_finn_onnx.onnx' at http://0.0.0.0:5902


In [8]:
#Doing the FINN-ONNX ececution here to compare against the onnx graph execution
in_X = np.array([0,0,0,0,0,0,0,0,0,0],dtype=np.float32)
#in_X = np.array([1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
#in_X = np.array([1,2,3,4,5,-6,-7,-8,-9,-10],dtype=np.float32)
#in_X = np.array([0,0,0,0,0,0,0,0,0,-0.7],dtype=np.float32)
in_X = in_X.reshape([10,1])
in_h_t_1 = np.zeros([20,1],dtype=np.float32).reshape([20,1])
input_dict = {}
input_dict["X"] = in_X
input_dict["h_t-1"] = in_h_t_1

output_dict = oxe.execute_onnx(finn_model, input_dict,return_full_exec_context=True)
output_dict

/scratch/users/khandelw/finn/deps/qonnx/src/qonnx/util/basic.py:300: UserWarning: The values of tensor qvGCMc can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(


{'h_t-1': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], dtype=float32),
 'X': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], dtype=float32),
 'scan_out': array([[0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392],
        [0.4980392]], dtype=float32),
 'out_m1': array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
   

In [9]:
#showInNetron('./onnx_lstm_model.onnx',localhost_url='xirxlabs53')#,localhost_url='xirxlabs53'
#showInNetron('./quantize-lstm-full-graph-test.onnx')#,localhost_url='xirxlabs53'

In [10]:
from finn.transformation.streamline import Streamline
showSrc(Streamline)

class Streamline(Transformation):
    """Apply the streamlining transform, see arXiv:1709.04060."""

    def apply(self, model):
        streamline_transformations = [
            ConvertSubToAdd(),
            ConvertDivToMul(),
            BatchNormToAffine(),
            ConvertSignToThres(),
            MoveMulPastMaxPool(),
            MoveScalarLinearPastInvariants(),
            AbsorbSignBiasIntoMultiThreshold(),
            MoveAddPastMul(),
            MoveScalarAddPastMatMul(),
            MoveAddPastConv(),
            MoveScalarMulPastMatMul(),
            MoveScalarMulPastConv(),
            MoveAddPastMul(),
            CollapseRepeatedAdd(),
            CollapseRepeatedMul(),
            MoveMulPastMaxPool(),
            AbsorbAddIntoMultiThreshold(),
            FactorOutMulSignMagnitude(),
            AbsorbMulIntoMultiThreshold(),
            Absorb1BitMulIntoMatMul(),
            Absorb1BitMulIntoConv(),
            RoundAndClipThresholds(),
        ]
        for tr

In [11]:
# graph = finn_model.graph
# node_ind = 0
# graph_modified = False
# for n in graph.node:
#     node_ind += 1
#     if n.op_type == "MatMul":
#         print(n.input)
#         matmul_weight_name = n.input[1]
#         print(n.input[1])
#         W = model.get_initializer(matmul_weight_name)
#         #print(W)
#         Wdt = model.get_tensor_datatype(matmul_weight_name)
#         assert W is not None, "Initializer for matmul weights is not set."
#W = finn_model.get_initializer('dql_ws')
#print(W)

In [13]:
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.fold_constants import FoldConstants
#streamlined_model = finn_model.transform(Streamline())
finn_model = finn_model.transform(InferShapes())
finn_model = finn_model.transform(FoldConstants())
finn_model = finn_model.transform(GiveUniqueNodeNames())
finn_model = finn_model.transform(GiveReadableTensorNames())
finn_model = finn_model.transform(InferDataTypes())
finn_model = finn_model.transform(RemoveStaticGraphInputs())

In [14]:
streamlined_model = finn_model.transform(Streamline())

In [15]:
streamlined_model.save("./streamlined_model.onnx")
showInNetron("./streamlined_model.onnx",localhost_url="xirxlabs53")

Stopping http://0.0.0.0:5902
Serving './streamlined_model.onnx' at http://0.0.0.0:5902
